In [1]:
import pandas as pd

In [2]:
tweet_data=pd.read_csv('Documents/metoo_tweets_dec2017[1]/metoo_tweets_dec2017.csv')
tweet_data.head()

,column_a,text,favorited,favoritecount,replytosn,created,truncated,replytosid,id,replytouid,statussource,screenname,retweetcount,isretweet,retweeted,longitude,latitude,location
0,1,American Harem.. #MeToo https://t.co/HjExLJdGuF,0.0,0,NaN,2017-11-29T23:59:00,0.0,NaN,9.360000e+17,NaN,"<a href=""http://instagram.com"" rel=""nofollow"">...",ahmediaTV,0.0,0.0,0.0,NaN,NaN,NaN
1,2,@johnconyersjr @alfranken why have you guys ...,0.0,0,johnconyersjr,2017-11-29T23:59:00,0.0,NaN,9.360000e+17,266149840,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",JesusPrepper74,0.0,0.0,0.0,NaN,NaN,NaN
2,3,Watched Megan Kelly ask Joe Keery this A.M. if...,0.0,0,NaN,2017-11-29T23:59:00,1.0,NaN,9.360000e+17,NaN,"<a href=""http://twitter.com/download/android"" ...",DemerisePotvin,0.0,0.0,0.0,NaN,NaN,NaN
3,4,Women have been talking about this crap the en...,0.0,0,NaN,2017-11-29T23:59:00,0.0,NaN,9.360000e+17,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",TheDawnStott,0.0,0.0,0.0,NaN,NaN,NaN
4,5,.@BetteMidler please speak to this sexual assa...,0.0,15,NaN,2017-11-29T23:59:00,0.0,NaN,9.360000e+17,NaN,"<a href=""http://twitter.com/#!/download/ipad"" ...",scottygirl2014,11.0,0.0,0.0,NaN,NaN,NaN


In [3]:
import re
import nltk
from nltk import trigrams
from collections import defaultdict

In [4]:
tweets = [
    "The #MeToo movement is a powerful statement against harassment.",
    "Women around the world are speaking out using #MeToo.",
    "#MeToo stories are heart-wrenching and inspiring at the same time."
]

In [5]:
def preprocess_tweet(tweet):
    # Remove hashtags, mentions, URLs, and special characters
    tweet = re.sub(r'@\w+', '', tweet)  # Remove mentions (@username)
    tweet = re.sub(r'http\S+', '', tweet)  # Remove URLs
    tweet = re.sub(r'#', '', tweet)  # Remove hashtag symbol, keep the tag text
    tweet = re.sub(r'[^A-Za-z\s]', '', tweet)  # Remove special characters (keep letters and spaces)
    tweet = tweet.lower()  # Convert to lowercase
    return tweet

# Apply preprocessing
processed_tweets = [preprocess_tweet(tweet) for tweet in tweets]

# Tokenize each tweet into words
tokenized_tweets = [nltk.word_tokenize(tweet) for tweet in processed_tweets]

In [6]:
# Generate trigrams from each tweet
tri_grams = [list(trigrams(tweet)) for tweet in tokenized_tweets]

# Flatten the list of trigrams (to have all trigrams in a single list)
flat_tri_grams = [trigram for tweet_trigrams in tri_grams for trigram in tweet_trigrams]


In [7]:
# Build a trigram model
model = defaultdict(lambda: defaultdict(lambda: 0))

# Count frequency of trigrams
for w1, w2, w3 in flat_tri_grams:
    model[(w1, w2)][w3] += 1

# Convert the counts to probabilities
for w1_w2 in model:
    total_count = float(sum(model[w1_w2].values()))
    for w3 in model[w1_w2]:
        model[w1_w2][w3] /= total_count


In [8]:
# Function to predict the next word
def predict_next_word(w1, w2):
    next_word = model.get((w1, w2), None)
    if next_word:
        predicted_word = max(next_word, key=next_word.get)
        return predicted_word
    else:
        return "No prediction available"

# Example usage (predict the word after 'the metoo')
print("Next Word:", predict_next_word('the', 'metoo'))


Next Word: movement


In [9]:
from collections import Counter
trigram_counts = Counter(flat_tri_grams)
print(trigram_counts.most_common(10)) 

[(('the', 'metoo', 'movement'), 1), (('metoo', 'movement', 'is'), 1), (('movement', 'is', 'a'), 1), (('is', 'a', 'powerful'), 1), (('a', 'powerful', 'statement'), 1), (('powerful', 'statement', 'against'), 1), (('statement', 'against', 'harassment'), 1), (('women', 'around', 'the'), 1), (('around', 'the', 'world'), 1), (('the', 'world', 'are'), 1)]
